In [42]:
from abc import ABC, abstractmethod

class Book:
    def __init__(self, book_id, title, author):
        self.book_id = book_id
        self.title = title
        self.author = author
        self.available = True

    def display_info(self):
        print(f"{self.book_id} | {self.title} | {self.author} | "
              f"{'Available' if self.available else 'Issued'}")

    def mark_issued(self):
        self.available = False

    def mark_returned(self):
        self.available = True


In [43]:
class Person(ABC):
    def __init__(self, member_id, name):
        self._member_id = member_id       # encapsulated attribute
        self.name = name

    def get_member_id(self):
        return self._member_id

    @abstractmethod
    def display_member_info(self):
        pass


In [44]:
class Member(Person):
    limit = 0   # overridden in child classes

    def __init__(self, member_id, name):
        super().__init__(member_id, name)
        self.issued_books = []

    def issue_book(self, book):
        if len(self.issued_books) >= self.limit:
            print(f"⚠ Borrow limit reached for {self.name}")
            return False
        self.issued_books.append(book)
        return True

    def return_book(self, book):
        if book in self.issued_books:
            self.issued_books.remove(book)
            return True
        return False

    def display_member_info(self):
        print(f"[Member] {self.name} (ID: {self.get_member_id()})")
        if not self.issued_books:
            print("  No books issued")
        else:
            print("  Books issued:")
            for b in self.issued_books:
                print(f"   - {b.title} ({b.book_id})")
        print()


In [45]:
class StudentMember(Member):
    limit = 3

    def display_member_info(self):
        print(f"[Student] {self.name} (ID: {self.get_member_id()})")
        if not self.issued_books:
            print("  No books issued")
        else:
            print("  Books issued:")
            for b in self.issued_books:
                print(f"   - {b.title} ({b.book_id})")
        print()


In [46]:
class TeacherMember(Member):
    limit = 5

    def display_member_info(self):
        print(f"[Teacher] {self.name} (ID: {self.get_member_id()})")
        if not self.issued_books:
            print("  No books issued")
        else:
            print("  Books issued:")
            for b in self.issued_books:
                print(f"   - {b.title} ({b.book_id})")
        print()


In [ ]:
class Library:
    def __init__(self):
        self.books = {}
        self.members = {}

    def add_book(self, book):
        self.books[book.book_id] = book

    def register_member(self, member):
        self.members[member.get_member_id()] = member

    def issue_book(self, member_id, book_id):
        if member_id not in self.members or book_id not in self.books:
            print("Invalid Member or Book ID")
            return

        member = self.members[member_id]
        book = self.books[book_id]

        if not book.available:
            print("Book already issued!")
            return

        if member.issue_book(book):
            book.mark_issued()
            print(f"{book.title} issued to {member.name}")

    def return_book(self, member_id, book_id):
        if member_id not in self.members or book_id not in self.books:
            print("Invalid Member or Book ID")
            return

        member = self.members[member_id]
        book = self.books[book_id]

        if member.return_book(book):
            book.mark_returned()
            print(f"{book.title} returned by {member.name}")
        else:
            print("Book not with this member")

    def display_all_books(self):
        print("\n--- LIBRARY BOOKS ---")
        for b in self.books.values():
            b.display_info()

    def display_all_members(self):
        print("\n--- LIBRARY MEMBERS ---")
        for m in self.members.values():
            m.display_member_info()


In [48]:
lib = Library()

# Books
lib.add_book(Book(100, "Python Basics", "Guido"))
lib.add_book(Book(200, "DSA", "Coreman"))
lib.add_book(Book(300, "OOP Concepts", "Stroustrup"))

# Members
lib.register_member(StudentMember(1, "Ali"))
lib.register_member(TeacherMember(2, "Sara"))

# Issue
lib.issue_book(1, 100)
lib.issue_book(1, 200)
lib.issue_book(2, 300)

# Display
lib.display_all_books()
lib.display_all_members()

# Return
lib.return_book(1, 100)
lib.display_all_members()


✔ Python Basics issued to Ali
✔ DSA issued to Ali
✔ OOP Concepts issued to Sara

--- LIBRARY BOOKS ---
100 | Python Basics | Guido | Issued
200 | DSA | Coreman | Issued
300 | OOP Concepts | Stroustrup | Issued

--- LIBRARY MEMBERS ---
[Student] Ali (ID: 1)
  Books issued:
   - Python Basics (100)
   - DSA (200)

[Teacher] Sara (ID: 2)
  Books issued:
   - OOP Concepts (300)

✔ Python Basics returned by Ali

--- LIBRARY MEMBERS ---
[Student] Ali (ID: 1)
  Books issued:
   - DSA (200)

[Teacher] Sara (ID: 2)
  Books issued:
   - OOP Concepts (300)

